In [1]:
import image_loader
import torch
csv_file = '../data/movie_data2.csv'
img_dir = '../data/MoviePosters'
pod = image_loader.PosterDataset(csv_file, img_dir, genres = None)

/Users/william/mambaforge/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch.utils.data import DataLoader

trainSize = int(len(pod) * 0.8)
testSize = len(pod) - trainSize

trainData, testData = torch.utils.data.random_split(pod, [trainSize, testSize])
def batch_function(raw_output):
    xs, ys = zip(*raw_output)
    batchxs = torch.nn.utils.rnn.pad_sequence(xs, batch_first=True)
    batchys = torch.Tensor(ys)
    return batchxs, batchys

trainDataLoader = DataLoader(trainData, batch_size = 10, shuffle=True, collate_fn=batch_function)
testDataLoader = DataLoader(testData, batch_size = 1, shuffle=True)


batchsize = 3


In [3]:
xs, ys = next(iter(trainDataLoader))
xs.shape

torch.Size([10, 3, 256, 256])

In [4]:
# for i in trainDataLoader:
#     print(i[0].dtype)

In [5]:
# download VGG19 model
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.vgg19_bn(pretrained=True, progress=True).features.to(device).eval()

/Users/william/mambaforge/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/william/mambaforge/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# check model architecture
print(model)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU(inplace=True)
  (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU(inplace=True)
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [7]:
# Initialize the model
# num_features = model.features[-1].out_channels  # Get the number of output channels of the last convolutional layer
# model.features.add_module('7', nn.Conv2d(num_features, 1, 1))  # Add a 1x1 convolutional layer to change the output size to 1
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
class MoviePredictor(torch.nn.Module):
    def __init__(self, vggmodel, linear_params):
        super().__init__()
        self.vggmodel = vggmodel
        self.predictor_1 = torch.nn.Linear(512 * 8 * 8, linear_params)
        self.out = torch.nn.Linear(linear_params, 1)
    def forward(self, input):
        vgg_out = self.vggmodel(input).reshape(input.shape[0], 512 * 8 * 8)
        output = self.out(self.predictor_1(vgg_out))
        return output

In [9]:
predictor_model = MoviePredictor(model, linear_params= 512)

In [10]:
predictor_model(pod[1][0][None,:,:,:])

tensor([[-0.4547]], grad_fn=<AddmmBackward0>)

In [11]:
predictor_model.vggmodel.requires_grad_(False)
predictor_optim = torch.optim.Adam(predictor_model.parameters())

In [14]:
# Train the model
from tqdm import tqdm
num_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    pbar = tqdm(trainDataLoader)
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = predictor_model(images)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        pbar.set_description(f"Loss: {loss}")
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

# Test the model
model.eval()
test_loss = 0.0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels.float())
        test_loss += loss.item()
print(f'Test Loss: {test_loss/len(test_loader)}')

Loss: 54.41765213012695:   4%|▎         | 103/2846 [02:08<56:56,  1.25s/it] 


KeyboardInterrupt: 